<a href="https://colab.research.google.com/github/arkalim/Tensorflow/blob/master/Preprocessing_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing functions for semantic segmenntation

## Convert a single annotated mask into one hot encoded mask

In [0]:
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
slim = tf.contrib.slim

def get_labels_from_annotation(annotation_tensor, class_labels):
    """Returns tensor of size (width, height, num_classes) derived from annotation tensor.
    The function returns tensor that is of a size (width, height, num_classes) which
    is derived from annotation tensor with sizes (width, height) where value at
    each position represents a class. The functions requires a list with class
    values like [0, 1, 2 ,3] -- they are used to derive labels. Derived values will
    be ordered in the same way as the class numbers were provided in the list. Last
    value in the aforementioned list represents a value that indicate that the pixel
    should be masked out. So, the size of num_classes := len(class_labels) - 1.
    Parameters
    ----------
    annotation_tensor : Tensor of size (width, height)
        Tensor with class labels for each element
    class_labels : list of ints
        List that contains the numbers that represent classes. Last
        value in the list should represent the number that was used
        for masking out.
    Returns
    -------
    labels_2d_stacked : Tensor of size (width, height, num_classes).
        Tensor with labels for each pixel.
    """

    # Last value in the classes list should show
    # which number was used in the annotation to mask out
    # the ambigious regions or regions that should not be
    # used for training.
    # TODO: probably replace class_labels list with some custom object
    valid_entries_class_labels = class_labels[:-1]

    # Stack the binary masks for each class
    labels_2d = list(map(lambda x: tf.equal(annotation_tensor, x), valid_entries_class_labels))

    # Perform the merging of all of the binary masks into one matrix
    labels_2d_stacked = tf.stack(labels_2d, axis=2)

    # Convert tf.bool to tf.float
    # Later on in the labels and logits will be used
    # in tf.softmax_cross_entropy_with_logits() function
    # where they have to be of the float type.
    labels_2d_stacked_float = tf.to_float(labels_2d_stacked)

    return labels_2d_stacked_float

W0617 10:58:15.271026 140583810160512 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
a = tf.constant([[0,1,1,0,1,0,10,2],
                 [0,2,1,0,2,0,2,1],
                 [1,2,1,0,1,0,1,2],
                 [0,0,1,0,1,0,0,2]])

print(a)

b = get_labels_from_annotation(a, [0,1,2,10,255])

print(b)

with tf.Session() as sess:
    print(b.eval())

Tensor("Const_1:0", shape=(4, 8), dtype=int32)
Tensor("ToFloat_1:0", shape=(4, 8, 4), dtype=float32)
[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]]


## Convert a batch of annotated masks into one hot encoded masks

In [0]:
def get_labels_from_annotation_batch(annotation_batch_tensor, class_labels):
    
    """Returns tensor of size (batch_size, width, height, num_classes) derived
    from annotation batch tensor. The function returns tensor that is of a size
    (batch_size, width, height, num_classes) which is derived from annotation tensor
    with sizes (batch_size, width, height) where value at each position represents a class.
    The functions requires a list with class values like [0, 1, 2 ,3] -- they are
    used to derive labels. Derived values will be ordered in the same way as
    the class numbers were provided in the list. Last value in the aforementioned
    list represents a value that indicate that the pixel should be masked out.
    So, the size of num_classes len(class_labels) - 1.
    Parameters
    ----------
    annotation_batch_tensor : Tensor of size (batch_size, width, height)
        Tensor with class labels for each element
    class_labels : list of ints
        List that contains the numbers that represent classes. Last
        value in the list should represent the number that was used
        for masking out.
    Returns
    -------
    batch_labels : Tensor of size (batch_size, width, height, num_classes).
        Tensor with labels for each batch.
    """

    # map the function 'get_labels_from_annotation()' to all the elements of 'annotation_batch_tensor'
    batch_labels = tf.map_fn(fn=lambda x: get_labels_from_annotation(annotation_tensor=x, class_labels=class_labels), elems=annotation_batch_tensor, dtype=tf.float32)

    return batch_labels


In [0]:
a = tf.constant([[[0,1,1,0,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,0,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,0,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]]])

print(a)

b = get_labels_from_annotation_batch(a, [0,1,2,255])

print(b)

with tf.Session() as sess:
    print(b.eval())

Tensor("Const_14:0", shape=(3, 4, 8), dtype=int32)
Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(3, 4, 8, 3), dtype=float32)
[[[[1. 0. 0.]
   [0. 1. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 0. 1.]
   [0. 0. 1.]]

  [[1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 0. 1.]
   [1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]]

  [[0. 1. 0.]
   [0. 0. 1.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [0. 0. 1.]]

  [[1. 0. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [1. 0. 0.]
   [0. 0. 1.]]]


 [[[1. 0. 0.]
   [0. 1. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 0. 1.]
   [0. 0. 1.]]

  [[1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 0. 1.]
   [1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]]

  [[0. 1. 0.]
   [0. 0. 1.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [0. 0. 1.]]

  [[1. 0. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [1. 0. 0.]
   [0

## Finding the position of pixels in the annotation masks which don't belong to the last class:255

In [0]:
def get_valid_entries_indices_from_annotation_batch(annotation_batch_tensor, class_labels):
    """
    Returns the indices of valid entries in the annotations.Output is a tensor of size 
    (num_valid_entries, 3) if the input is of shape [batch_size, width, height] and 
    if the input is of shape [width, height] then the output is of shape [num_valid_enteries,3]
    
    Valid entries are those entries in the annotations that don't belong to the last class: 255
    The last class is ambiguous and we dont't want them in our training case.
    
    Parameters
    ----------
    annotation_batch_tensor : Tensor of size (batch_size, width, height)
        Tensor with class labels for each batch
        
    class_labels : list of ints
        List that contains the numbers that represent classes. Last
        value in the list should represent the number that was used
        for masking out.
        
    Returns
    -------
    valid_labels_indices : 
        Tensor with indices of valid entries in row major form
        [row, col, depth] or [row, col] depending on the input
    """

    # Last value in the classes list should show
    # which number was used in the annotation to mask out
    # the ambigious regions or regions that should not be
    # used for training.

    mask_out_class_label = class_labels[-1]

    # Get binary mask for the pixels that we want to
    # use for training. We do this because some pixels
    # are marked as ambigious and we don't want to use
    # them for trainig to avoid confusing the model
    valid_labels_mask = tf.not_equal(annotation_batch_tensor, mask_out_class_label)

    # Search for the valid indices in row major form
    valid_labels_indices = tf.where(valid_labels_mask)

    return tf.to_int32(valid_labels_indices)


In [0]:
# A single annotation
a = tf.constant([[0,1,255,0,1,0,2,2],
                 [0,2,1,0,2,0,2,1],
                 [1,2,1,0,1,0,1,2],
                 [0,0,1,0,1,255,0,2]])

print(a)

b = get_valid_entries_indices_from_annotation_batch(a, [0,1,2,255])

print(b)

with tf.Session() as sess:
    print(b.eval().shape)
    print(b.eval())

Tensor("Const_21:0", shape=(4, 8), dtype=int32)
Tensor("ToInt32_5:0", shape=(?, 2), dtype=int32)
(30, 2)
[[0 0]
 [0 1]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [0 7]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]
 [1 5]
 [1 6]
 [1 7]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [2 4]
 [2 5]
 [2 6]
 [2 7]
 [3 0]
 [3 1]
 [3 2]
 [3 3]
 [3 4]
 [3 6]
 [3 7]]


In [0]:
# A batch of 3 annotations
a = tf.constant([[[0,1,1,0,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,0,1,0,2,2],
                  [0,2,255,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,225,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,225,0,1,0,0,2]]])

print(a)

b = get_valid_entries_indices_from_annotation_batch(a, [0,1,2,225])

print(b)

with tf.Session() as sess:
    print(b.eval().shape)
    print(b.eval())    

Tensor("Const_22:0", shape=(3, 4, 8), dtype=int32)
Tensor("ToInt32_6:0", shape=(?, 3), dtype=int32)
(94, 3)
[[0 0 0]
 [0 0 1]
 [0 0 2]
 [0 0 3]
 [0 0 4]
 [0 0 5]
 [0 0 6]
 [0 0 7]
 [0 1 0]
 [0 1 1]
 [0 1 2]
 [0 1 3]
 [0 1 4]
 [0 1 5]
 [0 1 6]
 [0 1 7]
 [0 2 0]
 [0 2 1]
 [0 2 2]
 [0 2 3]
 [0 2 4]
 [0 2 5]
 [0 2 6]
 [0 2 7]
 [0 3 0]
 [0 3 1]
 [0 3 2]
 [0 3 3]
 [0 3 4]
 [0 3 5]
 [0 3 6]
 [0 3 7]
 [1 0 0]
 [1 0 1]
 [1 0 2]
 [1 0 3]
 [1 0 4]
 [1 0 5]
 [1 0 6]
 [1 0 7]
 [1 1 0]
 [1 1 1]
 [1 1 2]
 [1 1 3]
 [1 1 4]
 [1 1 5]
 [1 1 6]
 [1 1 7]
 [1 2 0]
 [1 2 1]
 [1 2 2]
 [1 2 3]
 [1 2 4]
 [1 2 5]
 [1 2 6]
 [1 2 7]
 [1 3 0]
 [1 3 1]
 [1 3 2]
 [1 3 3]
 [1 3 4]
 [1 3 5]
 [1 3 6]
 [1 3 7]
 [2 0 0]
 [2 0 1]
 [2 0 2]
 [2 0 4]
 [2 0 5]
 [2 0 6]
 [2 0 7]
 [2 1 0]
 [2 1 1]
 [2 1 2]
 [2 1 3]
 [2 1 4]
 [2 1 5]
 [2 1 6]
 [2 1 7]
 [2 2 0]
 [2 2 1]
 [2 2 2]
 [2 2 3]
 [2 2 4]
 [2 2 5]
 [2 2 6]
 [2 2 7]
 [2 3 0]
 [2 3 1]
 [2 3 3]
 [2 3 4]
 [2 3 5]
 [2 3 6]
 [2 3 7]]


## Extract the valid pixels from logits and labels to apply softmax

In [0]:
def get_valid_logits_and_labels(annotation_batch_tensor, logits_batch_tensor, class_labels):
    """Returns two tensors of size (num_valid_entries, num_classes).
    The function converts annotation batch tensor input of the size
    (batch_size, height, width) into label tensor (batch_size, height,
    width, num_classes) and then selects only valid entries, resulting
    in tensor of the size (num_valid_entries, num_classes). The function
    also returns the tensor with corresponding valid entries in the logits
    tensor. Overall, two tensors of the same sizes are returned and later on
    can be used as an input into tf.softmax_cross_entropy_with_logits() to
    get the cross entropy error for each entry.
    Parameters
    ----------
    annotation_batch_tensor : Tensor of size (batch_size, width, height)
        Tensor with class labels for each batch
    logits_batch_tensor : Tensor of size (batch_size, width, height, num_classes)
        Tensor with logits. Usually can be achived after inference of fcn network.
    class_labels : list of ints
        List that contains the numbers that represent classes. Last
        value in the list should represent the number that was used
        for masking out.
    Returns
    -------
    (valid_labels_batch_tensor, valid_logits_batch_tensor) : Two Tensors of size (num_valid_eintries, num_classes).
        Tensors that represent valid labels and logits.
    """

    # Convert annotations into one hot encoded labels
    labels_batch_tensor = get_labels_from_annotation_batch(annotation_batch_tensor=annotation_batch_tensor, class_labels=class_labels)
    
    # get the indexes of valid pixels in annotations
    valid_batch_indices = get_valid_entries_indices_from_annotation_batch(annotation_batch_tensor=annotation_batch_tensor, class_labels=class_labels)

    # Select the valid pixels from labels
    valid_labels_batch_tensor = tf.gather_nd(params=labels_batch_tensor, indices=valid_batch_indices)

    # Select the valid pixels from logits 
    valid_logits_batch_tensor = tf.gather_nd(params=logits_batch_tensor, indices=valid_batch_indices)

    return valid_labels_batch_tensor, valid_logits_batch_tensor

In [0]:
# A batch of 3 annotations
a = tf.constant([[[0,1,1,0,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,0,1,0,2,2],
                  [0,2,255,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,225,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,225,0,1,0,0,2]]])

print(a)

# A batch of 3 annotations
b = tf.constant([[[0,1,1,0,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,0,1,0,2,2],
                  [0,2,255,0,255,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,1,0,1,0,0,2]],
                 
                 [[0,1,1,225,1,0,2,2],
                  [0,2,1,0,2,0,2,1],
                  [1,2,1,0,1,0,1,2],
                  [0,0,225,0,1,0,0,2]]])

b_categorical = get_labels_from_annotation_batch(b, [0,1,2,255])

print(b)

c,d = get_valid_logits_and_labels(a, b_categorical, [0,1,2,225])

print(c)
print(d)

with tf.Session() as sess:
    print(c.eval().shape)
    print(d.eval().shape) 

Tensor("Const_29:0", shape=(3, 4, 8), dtype=int32)
Tensor("Const_30:0", shape=(3, 4, 8), dtype=int32)
Tensor("GatherNd_6:0", shape=(?, 3), dtype=float32)
Tensor("GatherNd_7:0", shape=(?, 3), dtype=float32)
(94, 3)
(94, 3)
